## Data Preprocessing

This notebook illustrates steps required to preprocess [Wyscout Public Football Match Event Data](https://figshare.com/collections/Soccer_match_event_dataset/4415000/2) and make it ready for feeding into Expected Goals (xG) model. As an example case, data is preprocessed for English Premier League (EPL). 

### Libaries

In [1]:
# import required libraries
import os
import ast
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mplsoccer import Pitch, VerticalPitch

### Import Wyscout Public Match Event Dataset

Wyscout Public Match Event Dataset is available as `PublicWyscoutLoader` object in `socceraction` library.

In [2]:
# import wyscout public match event data loader from socceraction library
from socceraction.data.wyscout import PublicWyscoutLoader 

# load public wyscout data
wyscout_data = PublicWyscoutLoader()

To retrieve event data for all EPL matches, country specific (in our case, England) `competition_id` and `season_id` are required. 

In [3]:
# view all available competitions
wyscout_data.competitions()

,competition_id,season_id,country_name,competition_name,competition_gender,season_name
0,524,181248,Italy,Italian first division,male,2017/2018
1,364,181150,England,English first division,male,2017/2018
2,795,181144,Spain,Spanish first division,male,2017/2018
3,412,181189,France,French first division,male,2017/2018
4,426,181137,Germany,German first division,male,2017/2018
5,102,9291,International,European Championship,male,2016
6,28,10078,International,World Cup,male,2018


### EPL Data Preprocessing

#### Download Wyscout event data for EPL

Event data for each EPL match is downloaded, aggregated and saved into a single `events.csv` file. 

In [4]:
# England 17/18, competition_id = 364, season_id = 181150
epl_games = wyscout_data.games(competition_id = 364, season_id = 181150)["game_id"]

# download all premier league matches and save as a single .csv
events = pd.DataFrame()
for i in epl_games:
    df = wyscout_data.events(i)
    events = pd.concat([events, df])

# save as .csv file
events.to_csv('data/epl/events.csv', index = False) 

In [5]:
events.head()

,event_id,game_id,period_id,milliseconds,team_id,player_id,type_id,type_name,subtype_id,subtype_name,positions,tags
0,177959171,2499719,1,2758.649,1609,25413,8,Pass,85,Simple pass,"[{'y': 49, 'x': 49}, {'y': 78, 'x': 31}]",[{'id': 1801}]
1,177959172,2499719,1,4946.850,1609,370224,8,Pass,83,High pass,"[{'y': 78, 'x': 31}, {'y': 75, 'x': 51}]",[{'id': 1801}]
2,177959173,2499719,1,6542.188,1609,3319,8,Pass,82,Head pass,"[{'y': 75, 'x': 51}, {'y': 71, 'x': 35}]",[{'id': 1801}]
3,177959174,2499719,1,8143.395,1609,120339,8,Pass,82,Head pass,"[{'y': 71, 'x': 35}, {'y': 95, 'x': 41}]",[{'id': 1801}]
4,177959175,2499719,1,10302.366,1609,167145,8,Pass,85,Simple pass,"[{'y': 95, 'x': 41}, {'y': 88, 'x': 72}]",[{'id': 1801}]


In [7]:
events.shape

(643150, 12)

In total, there are 643150 events in 380 EPL matches.

#### Desirable formatting of raw events data

In [8]:
# upload 'events' dataframe that includes all events of all 380 EPL games
df = pd.read_csv('data/epl/events.csv')

In [9]:
# view columns
df.columns

Index(['event_id', 'game_id', 'period_id', 'milliseconds', 'team_id',
       'player_id', 'type_id', 'type_name', 'subtype_id', 'subtype_name',
       'positions', 'tags'],
      dtype='object')

In [10]:
# remove columns 'event id' and 'milliseconds'
rm_col_ind = np.r_[0, 3]
df = df.drop(columns = df.columns[rm_col_ind])

In [11]:
df.head()

,game_id,period_id,team_id,player_id,type_id,type_name,subtype_id,subtype_name,positions,tags
0,2499719,1,1609,25413,8,Pass,85,Simple pass,"[{'y': 49, 'x': 49}, {'y': 78, 'x': 31}]",[{'id': 1801}]
1,2499719,1,1609,370224,8,Pass,83,High pass,"[{'y': 78, 'x': 31}, {'y': 75, 'x': 51}]",[{'id': 1801}]
2,2499719,1,1609,3319,8,Pass,82,Head pass,"[{'y': 75, 'x': 51}, {'y': 71, 'x': 35}]",[{'id': 1801}]
3,2499719,1,1609,120339,8,Pass,82,Head pass,"[{'y': 71, 'x': 35}, {'y': 95, 'x': 41}]",[{'id': 1801}]
4,2499719,1,1609,167145,8,Pass,85,Simple pass,"[{'y': 95, 'x': 41}, {'y': 88, 'x': 72}]",[{'id': 1801}]


Columns `positions` and `tags` are in a list format but when saved as `.csv` file they are converted into a string format. When reading this `.csv` file, `literal_eval` function from `ast` library is used to transform strings back into lists.

In [12]:
# convert strings into python lists
df['tags'] = df['tags'].apply(ast.literal_eval)
df['positions'] = df['positions'].apply(ast.literal_eval)

# make 'type_name' and 'subtype_name' columns lowercase 
df['type_name'] = df['type_name'].str.lower()
df['subtype_name'] = df['subtype_name'].str.lower()

# create separate initial(start) and final(end) coordinates from 'positions' column
# if action has only 'start' coordinates set 'end' coordinates to 'nan'
df['x_start'] = df['positions'].apply(lambda x: x[0]['x'])
df['y_start'] = df['positions'].apply(lambda x: x[0]['y'])
df['x_end'] = df['positions'].apply(lambda x: x[1]['x'] if len(x) == 2 else np.nan)
df['y_end'] = df['positions'].apply(lambda x: x[1]['y'] if len(x) == 2  else np.nan)

In [13]:
df.head()

,game_id,period_id,team_id,player_id,type_id,type_name,subtype_id,subtype_name,positions,tags,x_start,y_start,x_end,y_end
0,2499719,1,1609,25413,8,pass,85,simple pass,"[{'y': 49, 'x': 49}, {'y': 78, 'x': 31}]",[{'id': 1801}],49,49,31.0,78.0
1,2499719,1,1609,370224,8,pass,83,high pass,"[{'y': 78, 'x': 31}, {'y': 75, 'x': 51}]",[{'id': 1801}],31,78,51.0,75.0
2,2499719,1,1609,3319,8,pass,82,head pass,"[{'y': 75, 'x': 51}, {'y': 71, 'x': 35}]",[{'id': 1801}],51,75,35.0,71.0
3,2499719,1,1609,120339,8,pass,82,head pass,"[{'y': 71, 'x': 35}, {'y': 95, 'x': 41}]",[{'id': 1801}],35,71,41.0,95.0
4,2499719,1,1609,167145,8,pass,85,simple pass,"[{'y': 95, 'x': 41}, {'y': 88, 'x': 72}]",[{'id': 1801}],41,95,72.0,88.0


To map tag ids in `tags` column to their definitions I use a separate `wyscout_tags.csv` file. This file is not a part of `PublicWyscoutLoader`, and it can be downloaded from [this link](https://figshare.com/articles/dataset/Mapping_of_tag_identifiers_to_tag_names/11743818?backTo=/collections/Soccer_match_event_dataset/4415000).

In [14]:
# import `tags.csv` file
tags = pd.read_csv('data/wyscout_tags.csv', sep = ';')

# make all descriptions lowercase
tags['Description'] = tags['Description'].str.lower()

# transform tags dataframe into dictionary
tags = dict(zip(tags['Tag'], tags['Description']))

# use dictionaries and list comprehensions to convert tags into tag ids and their descriptions 
df['tag_id'] = df['tags'].apply(lambda x: [value for d in x for value in d.values()])
df['tag_name'] = df['tag_id'].apply(lambda x: [tags[i] for i in x])

# drop redundant columns: 'positions', 'tags'
df.drop(columns = ['positions', 'tags'], inplace = True)

In [15]:
df.head()

,game_id,period_id,team_id,player_id,type_id,type_name,subtype_id,subtype_name,x_start,y_start,x_end,y_end,tag_id,tag_name
0,2499719,1,1609,25413,8,pass,85,simple pass,49,49,31.0,78.0,[1801],[accurate]
1,2499719,1,1609,370224,8,pass,83,high pass,31,78,51.0,75.0,[1801],[accurate]
2,2499719,1,1609,3319,8,pass,82,head pass,51,75,35.0,71.0,[1801],[accurate]
3,2499719,1,1609,120339,8,pass,82,head pass,35,71,41.0,95.0,[1801],[accurate]
4,2499719,1,1609,167145,8,pass,85,simple pass,41,95,72.0,88.0,[1801],[accurate]


For each event, there can be a multiple number of tags. Thus, these tags are stored as lists.

In [16]:
# rearrange columns using only required indices and passing them into np.r_
rearr_cols = np.r_[0:8, 12, 13, 8:12]
df = df.iloc[:, rearr_cols]

In [17]:
df.head()

,game_id,period_id,team_id,player_id,type_id,type_name,subtype_id,subtype_name,tag_id,tag_name,x_start,y_start,x_end,y_end
0,2499719,1,1609,25413,8,pass,85,simple pass,[1801],[accurate],49,49,31.0,78.0
1,2499719,1,1609,370224,8,pass,83,high pass,[1801],[accurate],31,78,51.0,75.0
2,2499719,1,1609,3319,8,pass,82,head pass,[1801],[accurate],51,75,35.0,71.0
3,2499719,1,1609,120339,8,pass,82,head pass,[1801],[accurate],35,71,41.0,95.0
4,2499719,1,1609,167145,8,pass,85,simple pass,[1801],[accurate],41,95,72.0,88.0


In [18]:
# save 'df' as 'refined_events.csv' dataframe
df.to_csv('data/epl/refined_events.csv', index = False)    

In [19]:
df.shape

(643150, 14)

#### Extract EPL shot event data 

In [20]:
# filter out all shot events for xG Model
# Note: free kicks are considered separate events and are not included (penalties are also part of free kicks)
shots = df[df['type_name'] == 'shot']

In [21]:
shots.head()

,game_id,period_id,team_id,player_id,type_id,type_name,subtype_id,subtype_name,tag_id,tag_name,x_start,y_start,x_end,y_end
46,2499719,1,1609,25413,10,shot,100,shot,"[101, 402, 201, 1205, 1801]","[goal, right foot, opportunity, position: goal...",88,41,0.0,0.0
62,2499719,1,1631,26150,10,shot,100,shot,"[401, 201, 1211, 1802]","[left foot, opportunity, position: out center ...",85,52,100.0,100.0
91,2499719,1,1631,14763,10,shot,100,shot,"[101, 403, 201, 1207, 1801]","[goal, head/body, opportunity, position: goal ...",96,52,100.0,100.0
128,2499719,1,1609,7868,10,shot,100,shot,"[401, 201, 1215, 1802]","[left foot, opportunity, position: out high le...",81,33,0.0,0.0
249,2499719,1,1609,7868,10,shot,100,shot,"[402, 201, 1205, 1801]","[right foot, opportunity, position: goal low l...",75,30,0.0,0.0


In [22]:
# save 'shots' dataframe as .csv file
shots.to_csv('data/epl/shots.csv', index = False)    

________________________________________________________________________________________________________________________
Since data retrieval and preprocessing steps are similar to what was done for EPL events dataset, all the above code was implemented in separate `.py` scripts for leagues in Spain, Germany, Italy and France. To see scripts, please follow this [link](https://github.com/faridmusayev/Expected_Goals_Model/tree/main/scripts).